## Setup

In [1]:
import os
# give googe drive the required permission
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Create a folder in your drive and add the name of that folder here. 
# For example, for the code below to run correctly, you need to have a folder named FoodRecSys in 'My Drive'.  
# The said folder will be your home directory for the rest of the project. 
# You will be able to save and read data from the folder. 

os.chdir("/content/drive/MyDrive/FoodRecSys/")
os.getcwd()

'/content/drive/MyDrive/FoodRecSys'

In [5]:
try:
  import pyspark 
except:
  !pip install pyspark==3.1.2
  import pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 212.4 MB 80 kB/s 
     |████████████████████████████████| 198 kB 57.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880769 sha256=e6e4a75bdbf7b086edea84ca69f386bf1971c27fce6f8dd27997d3c2d03f0618
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [6]:
try:
  import lenskit 
except:
  %pip install lenskit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 74 kB 2.7 MB/s 
     |████████████████████████████████| 256 kB 75.2 MB/s 
     |████████████████████████████████| 87 kB 7.5 MB/s 


In [7]:
from pyspark.sql import SparkSession
from pyspark.context import SparkContext

In [8]:
spark = SparkSession.builder.master("local").config('spark.ui.port', '4050').getOrCreate()

In [9]:
spark

## Imports

In [10]:
# import necessary libraries 
import pandas as pd
import numpy as np

In [11]:
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType,BooleanType,DateType,FloatType,StringType, ArrayType

In [12]:
# Import the required functions for ALS and estimating

In [13]:
from lenskit import batch, topn, util
from lenskit.algorithms import Recommender, als, item_knn as knn

## Read the data

To connect the data files with your google collab, download them and upload them to your home folder in your google drive. 

In [14]:
train_ratings_df = spark.read.parquet('/content/drive/MyDrive/food_recsys_project/Code Files/train/train_interaction_level_df.parquet', # Replace the given path with the path for your file
                                      header=True, 
                                      inferSchema=True)

In [15]:
train_ratings_df.show()

+-------+---------+------+--------------------+-----------+--------------------+-----+-------+--------------+----------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+--------------+
|user_id|recipe_id|rating|              review|review_date|                name|   id|minutes|contributor_id| submitted|                tags|           nutrition|n_steps|               steps|         description|         ingredients|n_ingredients|year_of_review|
+-------+---------+------+--------------------+-----------+--------------------+-----+-------+--------------+----------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+--------------+
| 152552|    29601|     5|Excellent. It was...| 2004-08-20|zippy cold spicy ...|29601|    270|         30367|2002-05-29|['time-to-make', ...|[277.2, 15.0, 13....|     12|['in a heavy 10-i...|shrimp drink in t...

In [16]:
test_ratings_all_df  = spark.read.parquet('/content/drive/MyDrive/food_recsys_project/Code Files/test/test_interaction_level_df.parquet', # Replace the given path with the path for your file
                                          header=True, 
                                          inferSchema=True)

In [17]:
!wget https://raw-recipes-clean-upgrad.s3.amazonaws.com/RAW_recipes_cleaned.csv -P /content/drive/MyDrive/FoodRecSys

--2022-10-07 03:25:02--  https://raw-recipes-clean-upgrad.s3.amazonaws.com/RAW_recipes_cleaned.csv
Resolving raw-recipes-clean-upgrad.s3.amazonaws.com (raw-recipes-clean-upgrad.s3.amazonaws.com)... 54.231.224.57
Connecting to raw-recipes-clean-upgrad.s3.amazonaws.com (raw-recipes-clean-upgrad.s3.amazonaws.com)|54.231.224.57|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 293745763 (280M) [text/csv]
Saving to: ‘/content/drive/MyDrive/FoodRecSys/RAW_recipes_cleaned.csv.1’

RAW_recipes_cleaned 100%[===================>] 280.14M  53.2MB/s    in 5.5s    

2022-10-07 03:25:08 (50.7 MB/s) - ‘/content/drive/MyDrive/FoodRecSys/RAW_recipes_cleaned.csv.1’ saved [293745763/293745763]



In [18]:
raw_recipes_df = spark.read.csv("/content/drive/MyDrive/FoodRecSys/RAW_recipes_cleaned.csv", # Replace the given path with the path for your file
                                header=True, 
                                inferSchema=True)

In [20]:
# Count the total number of ratings in the dataset
numerator = train_ratings_df.select("rating").count()

# Count the number of distinct userIds and distinct recipe_Ids
num_users = train_ratings_df.select("user_id").distinct().count()
num_recipes =  train_ratings_df.select("recipe_id").distinct().count()

# Set the denominator equal to the number of users multiplied by the number of movies
denominator = num_users * num_recipes

# Divide the numerator by the denominator
sparsity = (1.0 - (numerator *1.0)/denominator)*100
print("The training dataframe is ", "%.7f" % sparsity + "% empty.")

The training dataframe is  99.9829264% empty.


## Functions

In [21]:
def manual_recommendation_check (user_id):
  '''
  Given a user ID form the test dataset, this function will return the names of the recipes recommended to the user. 

  Initialize this function after the all_recs data frame is calculated. 

  Input user_id of a user from the test set as an integer.  
  Prints the names of recipes recommended to this user. 
  Returns nothing. 
  '''
  recs_user = all_recs_als[all_recs_als.user == user_id]
  recs_user_list = list(recs_user.item.values)
  recs_user_list = [x.item() for x in recs_user_list]
  display((raw_recipes_df.filter(F.col('id').isin(recs_user_list))
               .select("name")
               .collect()
               ))

## Model

#### Task 02 - Collaborative Filtering Model

Add the argument details in the algorithm initialization below to build the ALS model.

In [24]:
# Create ALS model
from pyspark.ml.recommendation import ALS
als = ALS(userCol= "user_id", # add the name of the column for users  
          itemCol= "recipe_id", # add the name of the column for recipes
          ratingCol= "rating", # add the name of the column for ratings 
          nonnegative = True, 
          implicitPrefs = False, 
          coldStartStrategy="drop"
         )

In [25]:
assert type(als) == pyspark.ml.recommendation.ALS

#### Training

In [26]:
#Fit the model to the 'train' dataset
model = als.fit(train_ratings_df)

## Prediction 

In [27]:
# use the model to create predictions for test data
test_predictions_unseen =  model.transform(test_ratings_all_df)# add a statment to tranform the test data

In [28]:
assert test_predictions_unseen.select(F.col("prediction"))

In [29]:
test_predictions_unseen.select(F.col("user_id"),
                               F.col("recipe_id"),
                               F.col("rating"),
                               F.col("prediction")
                              ).show(5)

+-------+---------+------+----------+
|user_id|recipe_id|rating|prediction|
+-------+---------+------+----------+
| 199020|    55265|     5|   4.60633|
| 369284|    76143|     5| 3.6038945|
| 224235|    89385|     5| 4.2184906|
| 385423|    95476|     4| 3.5020218|
| 538098|    95476|     5| 4.1667557|
+-------+---------+------+----------+
only showing top 5 rows



In [31]:
# Each user in the test set must have 10 predictions. 
# Use ALS model to get these predictions. 
# You can use the recommendForAllUsers() method. 

recommendations = model.recommendForAllUsers(10) # complete the code 

In [32]:
assert len(dict(recommendations.select(F.col("recommendations")).collect()[0][0])) == 10

In [34]:
recommendations.show()

+-------+---+--------------------+----------------+
|user_id|pos|recomended_recipe_id|predicted_rating|
+-------+---+--------------------+----------------+
|  28170|  0|              108417|       5.8745923|
|  28170|  1|              146652|       5.7722783|
|  28170|  2|               65833|       5.7687097|
|  28170|  3|              360099|       5.7478433|
|  28170|  4|              148469|        5.727963|
|  28170|  5|              181750|       5.6658654|
|  28170|  6|              325250|       5.6633644|
|  28170|  7|              236802|       5.6581197|
|  28170|  8|              320225|       5.6581197|
|  28170|  9|              325042|       5.6574197|
|  56680|  0|              362502|       7.0904617|
|  56680|  1|              211469|       6.5350037|
|  56680|  2|               76648|         6.42764|
|  56680|  3|              260960|       6.3290415|
|  56680|  4|               64539|        6.319501|
|  56680|  5|              272902|        6.301133|
|  56680|  6

In [33]:
# Transfroming the results by exploding the recommendations column

recommendations = (recommendations.select(F.col("user_id"),
                                          F.posexplode(F.col("recommendations")).alias("pos", "item")) 
                                  .select(F.col("user_id"),
                                          F.col("pos"), 
                                          F.col("item.recipe_id").alias("recomended_recipe_id"), 
                                          F.col("item.rating").alias("predicted_rating")))

## Evaluation

### Task 04 - Model Evaluation

#### 1. RMSE

Add the argument details to the evaluator function below to calculate the RMSE score of the ALS model. 

In [37]:
# Define evaluator as RMSE and print RMSE value
from pyspark.ml.evaluation import RegressionEvaluator
evaluator_seen = RegressionEvaluator(metricName="rmse", 
                                     labelCol= "rating",  # add the name of the column that has the ratings 
                                     predictionCol="prediction" # add the name of the column that has the predicted ratings. 
                                     ) 

In [40]:
RMSE = evaluator_seen.evaluate(test_predictions_unseen) # Use the evaluator to find the RMSE on the test set. 
print(RMSE)

1.403288862592361


#### Rank based Metrics

We will use the lenskit library to calculate the ranking-based matrics. The lenskit library is available in pandas only so we need to convert the data frames from PySpark dataframes to Pandas dataframes. 

In [41]:
all_recs_als = recommendations.toPandas()

In [44]:
all_recs_als.head()

,user_id,pos,recomended_recipe_id,predicted_rating
0,28170,0,108417,5.874592
1,28170,1,146652,5.772278
2,28170,2,65833,5.768710
3,28170,3,360099,5.747843
4,28170,4,148469,5.727963


In [45]:
# Rename the columns of to eunsre that they match the columns names as in the cell below. 

column_names = {'user_id': 'user', 'pos': 'rank', 'recomended_recipe_id': 'item','predicted_rating':'score'}# create a dictionary with current column names as keys and the intended column names as values.
all_recs_als = all_recs_als.rename(columns=column_names)

In [46]:
all_recs_als

,user,rank,item,score
0,28170,0,108417,5.874592
1,28170,1,146652,5.772278
2,28170,2,65833,5.768710
3,28170,3,360099,5.747843
4,28170,4,148469,5.727963
...,...,...,...,...
233615,1904821,5,9410,0.000000
233616,1904821,6,9970,0.000000
233617,1904821,7,10150,0.000000
233618,1904821,8,11440,0.000000


In [47]:
# Adding a column to make sure the we know these recommendations are from the ALS algorithm. 

all_recs_als["algorithm"] = "ALS"

In [50]:
test_predictions_unseen.select(F.col("user_id").alias("user"),
                                            F.col("recipe_id").alias("item"), 
                                            F.col("rating")).show()

+-------+------+------+
|   user|  item|rating|
+-------+------+------+
| 199020| 55265|     5|
| 369284| 76143|     5|
| 224235| 89385|     5|
| 385423| 95476|     4|
| 538098| 95476|     5|
|  80219| 95476|     5|
| 227652|134924|     5|
| 428885|141449|     4|
| 140132|146036|     5|
| 198154|146036|     4|
| 458462|146036|     5|
|1530179|146036|     5|
| 147095|146036|     5|
|1328641|146036|     5|
| 288146|146036|     5|
|1179225|146036|     4|
|1016844|146036|     5|
| 424680|195541|     5|
| 105804|226615|     0|
| 383346|232903|     4|
+-------+------+------+
only showing top 20 rows



In [64]:
# Convert the test dataset to pandas and ensure that it has the same column names as shown in the cell below. 
# Also, note there are only three columns in the rest data. You have to ensure your test data looks identical. 

test_data = (test_ratings_all_df.select(F.col("user_id").alias("user"),
                                            F.col("recipe_id").alias("item"), F.col("rating")).toPandas())

In [65]:
test_data

,user,item,rating
0,1802380878,35912,0
1,2001602879,40335,5
2,2758877,50348,0
3,199020,55265,5
4,369284,76143,5
...,...,...,...
18679,855082,438292,5
18680,1553277,447699,0
18681,2000072578,447699,5
18682,2775141,469503,5


In [67]:
# code to calculate the necessary metrics 
# the code below uses lenskit library to evaluate the rank metrics

rla = topn.RecListAnalysis()
rla.add_metric(topn.recip_rank)
rla.add_metric(topn.ndcg)
rla.add_metric(topn.dcg)
results = rla.compute(all_recs_als, test_data)
results.head()

nrecs  recip_rank  ndcg  dcg
algorithm user                                
ALS       28170      10         0.0   NaN  NaN
          56680      10         0.0   0.0  0.0
          108460     10         0.0   0.0  0.0
          139830     10         0.0   NaN  NaN
          198430     10         0.0   NaN  NaN

## Manual Prediction Checking

1. Why are ndcg and dcg nulls? EG: user 28170

In [68]:
test_data[test_data.user == 28170]

,user,item,rating


In [69]:
all_recs_als[all_recs_als.user == 28170]

,user,rank,item,score,algorithm
0,28170,0,108417,5.874592,ALS
1,28170,1,146652,5.772278,ALS
2,28170,2,65833,5.768710,ALS
3,28170,3,360099,5.747843,ALS
4,28170,4,148469,5.727963,ALS
5,28170,5,181750,5.665865,ALS
6,28170,6,325250,5.663364,ALS
7,28170,7,236802,5.658120,ALS
8,28170,8,320225,5.658120,ALS
9,28170,9,325042,5.657420,ALS


User ```28170``` does not appear in the test set. Hence cannot be evaluated. 

2. Why are all metrics 0 for specific users?

In [70]:
test_data[test_data.user == 56680]

,user,item,rating
446,56680,229831,5


In [71]:
all_recs_als[all_recs_als.user == 56680]

,user,rank,item,score,algorithm
10,56680,0,362502,7.090462,ALS
11,56680,1,211469,6.535004,ALS
12,56680,2,76648,6.427640,ALS
13,56680,3,260960,6.329041,ALS
14,56680,4,64539,6.319501,ALS
15,56680,5,272902,6.301133,ALS
16,56680,6,283977,6.215329,ALS
17,56680,7,346063,6.213429,ALS
18,56680,8,321589,6.198381,ALS
19,56680,9,260859,6.189267,ALS


User-recipe combination does not appear in the recommendations set. 

3. Are any non zero metrics? 

In [72]:
results[results.ndcg > 0]

nrecs  recip_rank      ndcg       dcg
algorithm user                                         
ALS       169430     10       0.200  0.020419  1.934264
          811179     10       0.125  0.208333  1.577324

In [73]:
results[results.dcg > 0]

nrecs  recip_rank      ndcg       dcg
algorithm user                                         
ALS       169430     10       0.200  0.020419  1.934264
          811179     10       0.125  0.208333  1.577324

In [74]:
results[results.recip_rank > 0]

nrecs  recip_rank      ndcg       dcg
algorithm user                                         
ALS       169430     10       0.200  0.020419  1.934264
          811179     10       0.125  0.208333  1.577324

There are few user recipe combination that has a corresponding value in the test set. Hence, we have only few non-zero value of evaluation metrics.


Manually, check one prediction. 

In [75]:
# use the function diclared earlier to print the recomendations for the user 653438
manual_recommendation_check(653438)

[Row(name='asian chicken noodle salad'),
 Row(name='baked spareribs aloha'),
 Row(name='briana s belt buster pot roast'),
 Row(name='ham  cheese and broccoli stuffed meatloaf'),
 Row(name='hillbilly beans'),
 Row(name='kittencal s oven roasted beef short ribs'),
 Row(name='mung bean salad'),
 Row(name='muskoka blueberry pie'),
 Row(name='roasted chickpeas with chard'),
 Row(name='swiss cheese and sausage deep dish')]

Of the recipes that have been recommended, few appear similar, and few do not. 

## Saving the models and predictions

In [ ]:
all_recs_als.to_csv("model_output/ALS/recommendation_als.csv", # modify the path 
                    index=False)

In [ ]:
model.save('model_output/ALS/ALS_model.model') # modify the path